In [1]:
def install_dependencies():
    ! rm -rf TinySQL || True
    ! git clone -b amir/check_errors https://github.com/withmartian/TinySQL.git
    ! cd TinySQL && pip install .
    ! pip install sqlparse
    ! pip install scikit-learn

install_dependencies()

Cloning into 'TinySQL'...
remote: Enumerating objects: 1662, done.
remote: Counting objects: 100% (353/353), done.
remote: Compressing objects: 100% (246/246), done.
remote: Total 1662 (delta 185), reused 177 (delta 107), pack-reused 1309 (from 2)
Receiving objects: 100% (1662/1662), 6.14 MiB | 20.14 MiB/s, done.
Resolving deltas: 100% (1029/1029), done.
Processing /home/amir_google/codes/TinySQL/notebooks/TinySQL
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for TinySQL: filename=TinySQL-1.3-py3-none-any.whl size=56481 sha256=1e8d386c718b8b49543c1b88a28f712f2c9d972b7b0ff358d7f4dbb0e1c91038
  Stored in directory: /var/tmp/pip-ephem-wheel-cache-pezhx5n6/wheels/40/db/00/2d9a687b5743d304e3823955f3f3e256a6ce41a86fc435e1d7
Successfully built TinySQL
  Attempting uninstall: TinySQL
    Found existing installation: TinySQL 1.3
    Uninstalling TinySQL-1.3:
      Successfully uninstalled Ti

In [11]:
from TinySQL.training_data.data_analyzer import get_errors
from TinySQL.classifiers.t5_classifier import train_t5_classifier
help(get_errors)

Help on function get_errors in module TinySQL.training_data.data_analyzer:

get_errors(max_seq_length=512, cs_num=3, model_num=1, syn=True, batch_size=32)



In [12]:
correct_and_errors_dataset = get_errors(cs_num=2)

Loading dataset withmartian/cs2_dataset_synonyms


Evaluating validation : 100%|████████████████████████████████████████████████████████████████████| 422/422 [04:45<00:00,  1.48it/s]


In [13]:
errors = correct_and_errors_dataset['errors']
correct_only = correct_and_errors_dataset['correct_predictions']

In [14]:
errors

Dataset({
    features: ['generated', 'expected', 'context', 'english', 'full_output'],
    num_rows: 1134
})

### Baseline use a T5 classifier

In [15]:
error_labels = [(sample["full_output"], 0) for sample in errors]
correct_labels = [(sample["full_output"], 1) for sample in correct_only]

all_labels = error_labels + correct_labels

In [16]:
accuracy, model = train_t5_classifier(all_labels)

Evaluating: 100%|████████████████████████████████████████████████████████████████████████████████| 338/338 [00:42<00:00,  7.90it/s]


In [17]:
accuracy

0.9070370370370371

In [20]:
len(error_labels)

1134

In [21]:
len(correct_labels)

12366